# Walmart Recruiting - Store Sales Forecasting -  Step 2: Baseline

Now that we have a first initial understanding of the problem we are trying to resolve, it is important to define
a metric for evaluating the success of the predictions. I will be using one of the most common metric for regression
problem: RMSE (Root-Mean-Square-Error)<br>

Another consideration before torturating the data to get more insights ;) it is a common practice to elaborate a 'common-sense' baseline. One naive approach is to take from the previous year the weekly sales for a given store/dept/week.

### Take away of the notebook
    . Defined the 'common-sense' baseline
    . Creating submission file
    . The baseline put me on the 1/3 of the competition pretty good for a baseline

In [2]:
import pandas as pd

# load data

In [3]:
df_train = pd.read_parquet('data/train.parquet')
df_test = pd.read_parquet('data/test.parquet')

# 'common-sense' baseline

In [13]:
# given a (store, dpt, week of the year) predict weekly sales with the corresponding value last year 
submission = pd.read_csv('data/sampleSubmission.csv')
submission['Store'] = (submission.Id.str.split('_', expand=True)[0]).astype(int)
submission['Dept'] = (submission.Id.str.split('_', expand=True)[1]).astype(int)
submission['Date'] = pd.to_datetime(submission.Id.str.split('_', expand=True)[2])
submission['Year'] = submission.Date.dt.year
submission['woy'] = submission.Date.dt.weekofyear

# extract sales from the previous year (same week number)
preds = df_train[['Store', 'Dept', 'Date', 'Weekly_Sales']].copy()
preds['Year'] = preds.Date.dt.year+1
preds['woy'] = preds.Date.dt.weekofyear

cols = submission.columns.drop(['Weekly_Sales'])
submission = submission[cols].merge(preds, how='left', on=['Store', 'Dept', 'Year', 'woy'])

# replace the rest by 0
submission.Weekly_Sales.fillna(0, inplace=True)
submission.Weekly_Sales.isnull().sum()

0

In [12]:
preds.head()

,Store,Dept,Date,Weekly_Sales,Id,Year,woy
0,1,1,2010-02-05,24924.50,1_1_2010-02-05,2011,5
1,1,1,2010-02-12,46039.49,1_1_2010-02-12,2011,6
2,1,1,2010-02-19,41595.55,1_1_2010-02-19,2011,7
3,1,1,2010-02-26,19403.54,1_1_2010-02-26,2011,8
4,1,1,2010-03-05,21827.90,1_1_2010-03-05,2011,9


# creating submission

In [7]:
# create submission file
cols = ['Id', 'Weekly_Sales']
submission[cols].to_csv('data/baseline.csv', index=False, header=True)